In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
import math
from decimal import Decimal, getcontext

In [74]:
genelist1=[]
genelist_start=[]
genelist_end=[]
for i in range(2,10):
    genelist1.append("GENE"+str(i))
    genelist_start.append("GENE"+str(i)+"-start")
    genelist_end.append("GENE"+str(i)+"-end")
alllist=[]
alllist+=genelist1
alllist+=genelist_start
alllist+=genelist_end

In [75]:
df=pd.read_csv("../../All-complementary region-genes.csv")
for i in alllist:
    df=df.drop(columns=i,axis=1)
df

,Race,chr,isMHC,start,end,len,genome browser,GENE Name,GENE-start,GENE-end
0,All,6,1,32453110,32577355,124245,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,HLA-DRB9,"32,459,820","32,460,089"
1,All,6,1,32453110,32577355,124245,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,HLA-DRB5,"32,517,353","32,530,287"
2,All,6,1,32453110,32577355,124245,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,HLA-DRB6,"32,552,713","32,560,002"
3,All,6,1,32453110,32577355,124245,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,RNU1-61P,"32,549,940","32,550,090"
4,All,6,1,32589647,32716541,126894,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,HLA-DQA1,"32,637,406","32,655,272"
5,All,6,1,32589647,32716541,126894,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,HLA-DQA1-AS1,"32,634,991","32,641,310"
6,All,6,1,32589647,32716541,126894,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,HLA-DQB1,"32,659,467","32,666,657"
7,All,6,1,32589647,32716541,126894,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,HLA-DQB1-AS1,"32,659,880","32,660,729"
8,All,6,1,32589647,32716541,126894,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,MTCO3P1,"32,706,144","32,706,803"
9,All,14,0,19806333,19976578,170245,https://www.ncbi.nlm.nih.gov/genome/gdv/browse...,OR4K6P,19847267,19848329


In [76]:
start_lists=df["GENE-start"].to_list()
end_lists=df["GENE-end"].to_list()
gene_names=df["GENE Name"].to_list()
ismhc=df["isMHC"].to_list()
chrs=df["chr"].to_list()
final_start_lists=[]
final_end_lists=[]
final_gene_names=[]
final_chrs=[]
final_ismhc=[]
for i in range(len(start_lists)-1):
    final_start_lists.append(start_lists[i])
    final_end_lists.append(end_lists[i])
    final_gene_names.append(gene_names[i])
    final_chrs.append(chrs[i])
    final_ismhc.append(ismhc[i])
final_chrs

[6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 7,
 7,
 1,
 1,
 1,
 1]

In [77]:
def confirm_precision(value):
    value_str = str(value)
    if '.' in value_str:
        print("value_str",value_str)
        print("len_value_str",len(value_str.split(".")[1]))
        return len(value_str.split(".")[1])
    else:
        return 0
    
def calculate_pvalue(pvalue):
    precision = confirm_precision(pvalue)
    getcontext().prec = precision
    p = Decimal(1) - Decimal(str(pvalue))
    formatted_p = "{:.{}f}".format(p, precision)  # 修改了这一行
    return formatted_p

In [89]:

all_mean_arr=-0.019077723390066942
all_std_arr=0.006128439002438663
left_value=all_mean_arr-2*all_std_arr
right_value=all_mean_arr+2*all_std_arr
print("left_value",left_value)
print("right_value",right_value)


left_value -0.03133460139494427
right_value -0.006820845385189615


In [79]:
allrecords=[]
for i in range(len(final_start_lists)):
    onerecord=[]
    onerecord.append(final_gene_names[i])
    onerecord.append(final_chrs[i])   
    onerecord.append(final_start_lists[i])
    onerecord.append(final_end_lists[i]) 
    filenames=os.listdir("../../csvfiles/")
    for filename in filenames:
        
        if filename.startswith("pos_prob_split_chr"+str(final_chrs[i])+"_"):
            print("chr",filename)
            if final_ismhc[i]==0:
                df_split=pd.read_csv("../../csvfiles/"+filename)  
                if final_start_lists[i]!="nan": 
                    string_number = final_start_lists[i]
                    cleaned_string_number = string_number.replace(",", "")
                    integer_number_start = int(cleaned_string_number)
                    string_number_end = final_end_lists[i]
                    cleaned_string_number_end = string_number_end.replace(",", "")
                    integer_number_end = int(cleaned_string_number_end)                 
                    allrigion_P_top_1000+=df_split[(df_split["Unnamed: 0"]>=integer_number_start) & (df_split["Unnamed: 0"]<=integer_number_end)]["P"].to_list()
                    if allrigion_P_top_1000:
                        sortedlist=sorted(allrigion_P_top_1000,reverse=True)
                        top_1000=sortedlist[:1000]
                        mean_1000_P=np.mean(top_1000)
                        std_1000_P=np.std(top_1000, ddof=1)
                        pvalue=1-round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),15)
                        onerecord+=[mean_1000_P,std_1000_P,pvalue]
                        allrecords.append(onerecord)
                        continue
                    else:
                        continue
            else:
                df_split=pd.read_csv("../../csvfiles/posmhc.csv") 
                if final_start_lists[i]!="nan": 
                    string_number = final_start_lists[i]
                    cleaned_string_number = string_number.replace(",", "")
                    integer_number_start = int(cleaned_string_number)
                    string_number_end = final_end_lists[i]
                    cleaned_string_number_end = string_number_end.replace(",", "")
                    integer_number_end = int(cleaned_string_number_end)                 
                    allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=integer_number_start) & (df_split["Unnamed: 0"]<=integer_number_end)]["P"].to_list()
                    if allrigion_P_top_1000:
                        sortedlist=sorted(allrigion_P_top_1000,reverse=True)
                        top_1000=sortedlist[:1000]
                        mean_1000_P=np.mean(top_1000)
                        std_1000_P=np.std(top_1000, ddof=1)
                        #np.set_printoptions(precision=15)
                        #stats_pvalue=stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr)
                        #stats_pvalue=round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),20)
                        #stats_pvalue_str = format(stats_pvalue, '.15e')
                        #print("mean_1000_P",mean_1000_P)
                        #print("stats_pvalue",stats_pvalue)
                        #pvalue=calculate_pvalue(stats_pvalue)
                        # pvalue=calculate_pvalue(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr))
                        #stats_pvalue=
                        #stats_pvalue=round(stats_pvalue,30)
                        #print("Stats_pvalue",stats_pvalue)
                        pvalue=1-round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),15)
                        print("pvalue",pvalue)
                        onerecord+=[mean_1000_P,std_1000_P,pvalue]
                        allrecords.append(onerecord)
                        continue
                    else:
                        continue
        else:
            continue
    

chr pos_prob_split_chr6_xat.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xaa.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xab.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xac.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xad.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xae.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xaf.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xai.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xaj.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xak.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xal.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xam.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xan.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xao.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xap.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xaq.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xar.csv
pvalue 0.287009967613053
chr pos_prob_split_chr6_xas.csv

In [ ]:
allrecords=[]
for i in range(len(final_start_lists)):
    onerecord=[]
    onerecord.append(final_gene_names[i])
    onerecord.append(final_chrs[i])   
    onerecord.append(final_start_lists[i])
    onerecord.append(final_end_lists[i]) 
    filenames=os.listdir("../../csvfiles/")
    for filename in filenames:
        
        if filename.startswith("pos_prob_split_chr"+str(final_chrs[i])+"_"):
            print("chr",filename)
            if final_ismhc[i]==0:
                df_split=pd.read_csv("../../csvfiles/"+filename)  
                if final_start_lists[i]!="nan": 
                    string_number = final_start_lists[i]
                    cleaned_string_number = string_number.replace(",", "")
                    integer_number_start = int(cleaned_string_number)
                    string_number_end = final_end_lists[i]
                    cleaned_string_number_end = string_number_end.replace(",", "")
                    integer_number_end = int(cleaned_string_number_end)                 
                    allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=integer_number_start) & (df_split["Unnamed: 0"]<=integer_number_end)]["P"].to_list()
                    if allrigion_P_top_1000:
                        sortedlist=sorted(allrigion_P_top_1000,reverse=True)
                        top_1000=sortedlist[:1000]
                        mean_1000_P=np.mean(top_1000)
                        std_1000_P=np.std(top_1000, ddof=1)
                        #pvalue=1-round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),15)   
                       
                        # stats_pvalue=round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),20)
                        # print("mean_1000_P",mean_1000_P)
                        
                        
                        # print("stats_pvalue",stats_pvalue)
                        # pvalue=calculate_pvalue(stats_pvalue)
                        # print("pvalue",pvalue)
                        # if math.isclose(0.0,pvalue):
                        #     print(pvalue)
                        # stats_pvalue=stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr)
                        #stats_pvalue=round(stats_pvalue,30)
                        #print("Stats_pvalue",stats_pvalue)
                        pvalue=1-round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),15)
                        onerecord+=[mean_1000_P,std_1000_P,pvalue]
                        allrecords.append(onerecord)
                        continue
                    else:
                        continue
            else:
                df_split=pd.read_csv("../../csvfiles/posmhc.csv") 
                if final_start_lists[i]!="nan": 
                    string_number = final_start_lists[i]
                    cleaned_string_number = string_number.replace(",", "")
                    integer_number_start = int(cleaned_string_number)
                    string_number_end = final_end_lists[i]
                    cleaned_string_number_end = string_number_end.replace(",", "")
                    integer_number_end = int(cleaned_string_number_end)                 
                    allrigion_P_top_1000=df_split[(df_split["Unnamed: 0"]>=integer_number_start) & (df_split["Unnamed: 0"]<=integer_number_end)]["P"].to_list()
                    if allrigion_P_top_1000:
                        sortedlist=sorted(allrigion_P_top_1000,reverse=True)
                        top_1000=sortedlist[:1000]
                        mean_1000_P=np.mean(top_1000)
                        std_1000_P=np.std(top_1000, ddof=1)
                        #np.set_printoptions(precision=15)
                        #stats_pvalue=stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr)
                        #stats_pvalue=round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),20)
                        #stats_pvalue_str = format(stats_pvalue, '.15e')
                        #print("mean_1000_P",mean_1000_P)
                        #print("stats_pvalue",stats_pvalue)
                        #pvalue=calculate_pvalue(stats_pvalue)
                        # pvalue=calculate_pvalue(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr))
                        #stats_pvalue=
                        #stats_pvalue=round(stats_pvalue,30)
                        #print("Stats_pvalue",stats_pvalue)
                        pvalue=1-round(stats.norm.cdf(mean_1000_P,all_mean_arr, all_std_arr),15)
                        print("pvalue",pvalue)
                        onerecord+=[mean_1000_P,std_1000_P,pvalue]
                        allrecords.append(onerecord)
                        continue
                    else:
                        continue
        else:
            continue
    

In [2]:
# all_mean_arr=-0.019077723390066942
# all_std_arr=0.006128439002438663
print(1-round(stats.norm.cdf(-0.018984	,-0.019077723390066942, 0.006128439002438663),15))

0.49389913720034995


In [81]:
#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    dataframe(dictforDF).to_csv("../../formalcsvs/form2_zhengwen_2023_05_04.csv")
    return dataframe(dictforDF)

In [82]:
finalrecords=[]
for allrecord in allrecords:
    finalrecords.append(tuple(allrecord))

In [83]:
finalrecords=list(set(finalrecords))
unique_nested_list = [list(item) for item in finalrecords]


In [84]:
print(len(finalrecords))
print(len(unique_nested_list))

24
24


In [85]:
unique_nested_list

[['HLA-DQB1-AS1',
  6,
  '32,659,880',
  '32,660,729',
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.018983542361729396,
  0.10552774092609964,
  0.49386934987571396,
  -0.01898354236172939

In [86]:
columns=["GENE name","chr","GENE start","GENE end","Average of top 1000 P","Std of top 1000 P","P-value of top 1000 p"]

In [87]:
generatedf(columns,unique_nested_list)

,GENE name,chr,GENE start,GENE end,Average of top 1000 P,Std of top 1000 P,P-value of top 1000 p
0,HLA-DQB1-AS1,6,"32,659,880","32,660,729",-0.018984,0.105528,4.938693e-01
1,LOC124903278,14,"19,902,461","19,948,734",0.026657,0.100861,4.196643e-14
2,TMA16P2,1,"45,845,343","45,847,719",-0.004966,0.053339,1.065021e-02
3,OR4K4P,14,"19,905,712","19,906,644",-0.018044,0.023642,4.329961e-01
4,OR4K1,14,"19,920,263","19,936,757",-0.004301,0.080621,7.951014e-03
5,OR4K3,14,"19,868,197","19,869,173",0.000050,0.110962,9.006950e-04
6,RNU1-61P,6,"32,549,940","32,550,090",0.032751,0.162795,0.000000e+00
7,MTCO3P1,6,"32,706,144","32,706,803",0.019481,0.109457,1.568840e-10
8,\tOR4K5,14,"19,920,607","19,921,578",-0.030253,0.033357,9.658833e-01
9,POT1,7,"124,822,386","124,929,825",0.039317,0.084435,0.000000e+00
